In [60]:
import psycopg2
from scipy import stats as kl

In [61]:
dimension_attributes = ['workclass','education','education_num','occupation','relationship','race','sex','native_country','salary_range']
measure_attributes = ['age','fnlwgt','capital_gain','capital_loss','hours_per_week']
aggregate_functions = ['sum', 'avg', 'max', 'min', 'count']

In [62]:
conn = psycopg2.connect()

In [63]:
def generate_views():
    
    aggregate_views = []
    for a in dimension_attributes:
        for m in measure_attributes:
            for f in aggregate_functions:
                aggregate_views.append((a,m,f))
    return aggregate_views

def normalize(qResult1, qResult2):
    
    dict1 = {}
    dict2 = {}
    
    for i in qResult1:
        
        dict1[i[0]] = float(i[1])
        if i[0] not in qResult2:
            dict2[i[0]] = 1e-9
        else:
            dict2[i[0]] = float(qResult2[i[0]])
        
    for i in qResult2:
        
        if i[0] not in dict1:
            dict1[i[0]] = 1e-9
        
        if i[0] not in dict2:
            dict2[i[0]] = float(i[1])


        
    if sum(dict1.values()) != 0:
        factor=1.0/sum(dict1.values())
        for k in dict1:
            dict1[k] = dict1[k]*factor
            
    if sum(dict2.values()) != 0:   
        factor=1.0/sum(dict2.values())
        for k in dict2:
            dict2[k] = dict2[k]*factor
    
    return dict1, dict2
    
def calc_utility(res1, res2):
    
    temp1 = []
    temp2 = []
    for i in res1:
        temp1.append(res1[i])
    
    for i in res2:
        temp2.append(res2[i])
        
    return kl.entropy(temp1, temp2)
    

def execute(target_view, reference_view):
    
    cur1 = conn.cursor()
    cur2 = conn.cursor()
    
    cur1.execute(target_view)
    cur2.execute(reference_view)
    
    res1, res2 = normalize(cur1.fetchall(), cur2.fetchall())
    
    return calc_utility(res1, res2)
    
    
def execute_queries(aggregate_views):
    
    results = []
    for a,m,f in aggregate_views:
        target_view = 'select ' + a +', ' + f + '(' + m + ')' + ' from married_adults3 group by ' + a + ';'
        reference_view = 'select ' + a +',' + f + '(' + m + ')' + 'from unmarried_adults3 group by ' + a + ';'
        utility_measure = execute(target_view, reference_view)
        results.append([(a,m,f), utility_measure])
    
    return results

def get_topK(results, k):
    
    results.sort(key = lambda x: x[1], reverse = True)
    return results[0:k]

In [65]:
k = 5
aggregate_views = generate_views()
results = execute_queries(aggregate_views)
topK_results = get_topK(results, k)
print(topK_results)

[[('workclass', 'fnlwgt', 'sum'), 0.8760182291179579], [('workclass', 'age', 'count'), 0.8501510619402023], [('workclass', 'fnlwgt', 'count'), 0.8501510619402023], [('workclass', 'age', 'sum'), 0.8008858374717213], [('workclass', 'capital_gain', 'min'), nan]]
